In [75]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor



from sklearn.linear_model import RANSACRegressor

import seaborn as sns
import xgboost as xgb
import lightgbm as lgb
import catboost as cb

import tensorflow as tf
import keras
from keras import layers

In [60]:
df = pd.read_csv('dataset_regression/parkinsons_updrs_cleaned.data')

In [61]:
df.head()

,age,sex,motor_UPDRS,Jitter_pct,Jitter_Abs,Jitter_RAP,Jitter_PPQ5,Jitter_DDP,Shimmer,Shimmer_dB,Shimmer_APQ3,Shimmer_APQ5,Shimmer_APQ11,Shimmer_DDA,NHR,HNR,RPDE,DFA,PPE
0,72,0,28.199,0.00662,0.000034,0.00401,0.00317,0.01204,0.02565,0.230,0.01438,0.01309,0.01662,0.04314,0.014290,21.640,0.41888,0.54842,0.16006
1,72,0,28.447,0.00300,0.000017,0.00132,0.00150,0.00395,0.02024,0.179,0.00994,0.01072,0.01689,0.02982,0.011112,27.183,0.43493,0.56477,0.10810
2,72,0,28.695,0.00481,0.000025,0.00205,0.00208,0.00616,0.01675,0.181,0.00734,0.00844,0.01458,0.02202,0.020220,23.047,0.46222,0.54405,0.21014
3,72,0,28.905,0.00528,0.000027,0.00191,0.00264,0.00573,0.02309,0.327,0.01106,0.01265,0.01963,0.03317,0.027837,24.445,0.48730,0.57794,0.33277
4,72,0,29.187,0.00335,0.000020,0.00093,0.00130,0.00278,0.01703,0.176,0.00679,0.00929,0.01819,0.02036,0.011625,26.126,0.47188,0.56122,0.19361


In [62]:
df.shape

(5070, 19)

In [52]:
X = df.drop(['motor_UPDRS'], axis=1)
y = df['motor_UPDRS']

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)   

In [54]:
print("Overall distribution:")
df['motor_UPDRS'].describe()    

Overall distribution:


count    5009.000000
mean       21.509002
std         7.976287
min         5.037700
25%        15.000000
50%        21.192000
75%        27.689000
max        39.511000
Name: motor_UPDRS, dtype: float64

In [55]:

print("\nTraining set distribution:")
y_train.describe()  



Training set distribution:


count    4007.000000
mean       21.495811
std         7.989770
min         5.037700
25%        15.000000
50%        21.107000
75%        27.663000
max        39.511000
Name: motor_UPDRS, dtype: float64

In [63]:

print("\nTest set distribution:")
y_test.describe()   



Test set distribution:


count    1002.000000
mean       21.561754
std         7.925888
min         5.137500
25%        15.000000
50%        21.346500
75%        27.729000
max        39.511000
Name: motor_UPDRS, dtype: float64

In [64]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [74]:
models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree": DecisionTreeRegressor(random_state=42),
    "Random Forest": RandomForestRegressor(random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(random_state=42)
}


NameError: name 'DecisionTreeRegressor' is not defined